# Historical Data for Analysis
## How to Use the Data

* Luno and Binance only
* Order book top 1 level for both ask and bid
* Spot price between BTC and ZAR
* Real-time data, so all changes in levels are recorded
* From 2020-06-15T00:00:00 to 2020-07-17T00:00:00 (32 days from Monday until Thursday)
* Could not get more due to free API limits

In [64]:
import json
import pandas as pd

In [65]:
with open('luno_binance_spot_btc_zar.json', 'r') as f:
    luno_binance_data = json.load(f)

luno_btc_zar_data = pd.DataFrame(luno_binance_data['LUNO_SPOT_BTC_ZAR'])
binance_btc_zar_data = pd.DataFrame(luno_binance_data['BINANCE_SPOT_BTC_ZAR'])

In [66]:
luno_btc_zar_data.head()

,time,asks_price,asks_size,bids_price,bids_size
0,2020-06-15T00:02:19.5214332Z,170386.0,0.534161,170385.0,0.076187
1,2020-06-15T00:02:34.7003021Z,170386.0,0.455860,170385.0,2.500000
2,2020-06-15T00:03:45.2280492Z,170386.0,0.534161,170385.0,1.949240
3,2020-06-15T00:04:36.1349999Z,170386.0,0.455860,170385.0,2.500000
4,2020-06-15T00:05:10.9274555Z,170386.0,0.534161,170385.0,2.500000


In [67]:
binance_btc_zar_data.head()

,time,asks_price,asks_size,bids_price,bids_size
0,2020-06-15T00:00:13.4120000Z,171542.0,0.089691,170523.0,0.003790
1,2020-06-15T00:00:21.0070000Z,171542.0,0.089691,170523.0,0.003790
2,2020-06-15T00:00:27.5090000Z,171542.0,0.089691,170523.0,0.003790
3,2020-06-15T00:00:28.5090000Z,171542.0,0.089691,170523.0,0.003790
4,2020-06-15T00:00:29.8080000Z,171542.0,0.089691,170514.0,0.117292


## Getting the Data for BTC-ZAR

In [2]:
# set-up

import requests

api_key = '93E4B0FE-5A23-456C-A8E2-FE04A9EC22D7'
email = 'tsofgygbbxnzwrbtoi@ttirv.org'

In [6]:
symbol_id_list = ['LUNO_SPOT_BTC_ZAR',
    '1BTCXE_SPOT_BTC_ZAR',
    'ALTCOINTRADER_SPOT_BTC_ZAR',
    'ICE3X_SPOT_BTC_ZAR',
    'STELLARPORT_SPOT_BTC_ZAR',
    'BISQ_SPOT_BTC_ZAR',
    'BINANCE_SPOT_BTC_ZAR',
    'ARTISTURBA_SPOT_BTC_ZAR',
    'STELLARPORT_SPOT_BTC_ZART',
    'OVEX_SPOT_BTC_ZAR']

historical_data_1d_dct = {}

In [7]:
for symbol_id in symbol_id_list:
    time_start = '2020-07-15T00:00:00'
    time_end = '2020-07-16T00:00:00'
    limit = '100000'
    limit_levels = '1'

    url = 'https://rest.coinapi.io/v1/orderbooks/' + \
        symbol_id + '/history?time_start=' + time_start + \
        '&time_end=' + time_end + '&limit=' + limit + '&limit_levels=' + limit_levels
    headers = {'X-CoinAPI-Key' : api_key}
    response = requests.get(url, headers=headers)

    historical_data_1d_dct[symbol_id] = response.json()

Find the activity at different exchanges

In [9]:
for symbol_id in symbol_id_list:
    print(symbol_id + ': ' + str(len(historical_data_1d_dct[symbol_id])))

LUNO_SPOT_BTC_ZAR: 960
1BTCXE_SPOT_BTC_ZAR: 0
ALTCOINTRADER_SPOT_BTC_ZAR: 0
ICE3X_SPOT_BTC_ZAR: 760
STELLARPORT_SPOT_BTC_ZAR: 0
BISQ_SPOT_BTC_ZAR: 141
BINANCE_SPOT_BTC_ZAR: 793
ARTISTURBA_SPOT_BTC_ZAR: 0
STELLARPORT_SPOT_BTC_ZART: 0
OVEX_SPOT_BTC_ZAR: 0


It seems that Luno is the most active and there are typically less than 1000 updates per day. So we can easily get 4 weeks of data for initial analysis.

In [10]:
# we choose 2020-06-15T00:00:00 as start - Monday about 5 weeks ago
# we choose 2020-07-18T00:00:00 as end - start of latest Saturday

historical_data_33d_dct = {}

for symbol_id in symbol_id_list:
    time_start = '2020-06-15T00:00:00'
    time_end = '2020-07-18T00:00:00'
    limit = '100000'
    limit_levels = '1'

    url = 'https://rest.coinapi.io/v1/orderbooks/' + \
        symbol_id + '/history?time_start=' + time_start + \
        '&time_end=' + time_end + '&limit=' + limit + '&limit_levels=' + limit_levels
    headers = {'X-CoinAPI-Key' : api_key}
    response = requests.get(url, headers=headers)

    historical_data_33d_dct[symbol_id] = response.json()

Check how much updates we got - was there an overflow (over 100000)?

In [11]:
for symbol_id in symbol_id_list:
    print(symbol_id + ': ' + str(len(historical_data_33d_dct[symbol_id])))

LUNO_SPOT_BTC_ZAR: 58975
1BTCXE_SPOT_BTC_ZAR: 0
ALTCOINTRADER_SPOT_BTC_ZAR: 0
ICE3X_SPOT_BTC_ZAR: 5
STELLARPORT_SPOT_BTC_ZAR: 0
BISQ_SPOT_BTC_ZAR: 1156
BINANCE_SPOT_BTC_ZAR: 100000
ARTISTURBA_SPOT_BTC_ZAR: 5
STELLARPORT_SPOT_BTC_ZART: 5
OVEX_SPOT_BTC_ZAR: 0


It looks like there was an overflow for Binance - so only get 2-3 weeks now

In [14]:
# we choose 2020-06-29T00:00:00 as start - Monday about 3 weeks ago
# we choose 2020-07-18T00:00:00 as end - start of latest Saturday

historical_data_19d_dct = {}

for symbol_id in symbol_id_list:
    time_start = '2020-06-29T00:00:00'
    time_end = '2020-07-18T00:00:00'
    limit = '100000'
    limit_levels = '1'

    url = 'https://rest.coinapi.io/v1/orderbooks/' + \
        symbol_id + '/history?time_start=' + time_start + \
        '&time_end=' + time_end + '&limit=' + limit + '&limit_levels=' + limit_levels
    headers = {'X-CoinAPI-Key' : api_key}
    response = requests.get(url, headers=headers)

    historical_data_19d_dct[symbol_id] = response.json()

In [15]:
for symbol_id in symbol_id_list:
    print(symbol_id + ': ' + str(len(historical_data_19d_dct[symbol_id])))

LUNO_SPOT_BTC_ZAR: 5
1BTCXE_SPOT_BTC_ZAR: 5
ALTCOINTRADER_SPOT_BTC_ZAR: 5
ICE3X_SPOT_BTC_ZAR: 5
STELLARPORT_SPOT_BTC_ZAR: 5
BISQ_SPOT_BTC_ZAR: 5
BINANCE_SPOT_BTC_ZAR: 5
ARTISTURBA_SPOT_BTC_ZAR: 5
STELLARPORT_SPOT_BTC_ZART: 5
OVEX_SPOT_BTC_ZAR: 5


In [16]:
historical_data_19d_dct['LUNO_SPOT_BTC_ZAR']

{'error': 'Too many requests - You have exceeded your API key last 24 hour requests executed limit, please wait for new requests or contact support for upgrading your existing plan.',
 'faq_0': 'Q: How requests are calculated? A: The request is a single API call to our REST API without `limit` parameter. Or with `limit` parameter, the request is counted every 100 data points returned by the API.',
 'faq_1': 'Q: How I can monitor number of requests left? A: Every authenticated API call providing information about the current state of the limits in the HTTP response headers, more info here: https://docs.coinapi.io/#request-limits',
 'faq_2': "Q: I'm on Professional plan. How I can have more requests? A: Please contact support at the https://www.coinapi.io/support for more information.",
 'faq_3': "Q: I'm on free plan and I didn't executed 100 requests yet and got 429 error. A: Free plan contains additional limitations to prevent fraud and usage of the multiple free API keys to obey the l

It seems that now we have used the free number of requests, so we will need to use what we managed to get.

Look at the data we managed to get

In [25]:
for symbol_id in symbol_id_list:
    if len(historical_data_33d_dct[symbol_id]) > 0:
        print(symbol_id + ': ' + str(len(historical_data_33d_dct[symbol_id])))

LUNO_SPOT_BTC_ZAR: 58975
ICE3X_SPOT_BTC_ZAR: 5
BISQ_SPOT_BTC_ZAR: 1156
BINANCE_SPOT_BTC_ZAR: 100000
ARTISTURBA_SPOT_BTC_ZAR: 5
STELLARPORT_SPOT_BTC_ZART: 5


In [32]:
historical_data_33d_dct['ICE3X_SPOT_BTC_ZAR']

{'error': 'Too many requests - You have exceeded your API key last 24 hour requests executed limit, please wait for new requests or contact support for upgrading your existing plan.',
 'faq_0': 'Q: How requests are calculated? A: The request is a single API call to our REST API without `limit` parameter. Or with `limit` parameter, the request is counted every 100 data points returned by the API.',
 'faq_1': 'Q: How I can monitor number of requests left? A: Every authenticated API call providing information about the current state of the limits in the HTTP response headers, more info here: https://docs.coinapi.io/#request-limits',
 'faq_2': "Q: I'm on Professional plan. How I can have more requests? A: Please contact support at the https://www.coinapi.io/support for more information.",
 'faq_3': "Q: I'm on free plan and I didn't executed 100 requests yet and got 429 error. A: Free plan contains additional limitations to prevent fraud and usage of the multiple free API keys to obey the l

It looks that only Luno and Binance are useful at all - so save these for the analysis

In [34]:
data_to_save = {'LUNO_SPOT_BTC_ZAR': historical_data_33d_dct['LUNO_SPOT_BTC_ZAR'],
    'BINANCE_SPOT_BTC_ZAR': historical_data_33d_dct['BINANCE_SPOT_BTC_ZAR']}

In [35]:
import json

In [36]:
with open('zar_data.json', 'w') as f:
    json.dump(data_to_save, f)

In [43]:
historical_data_33d_dct['LUNO_SPOT_BTC_ZAR'][-5:]

[{'symbol_id': 'LUNO_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T23:54:08.5063594Z',
  'time_coinapi': '2020-07-17T23:54:08.5063594Z',
  'asks': [{'price': 164317.0, 'size': 0.8113}],
  'bids': [{'price': 164020.0, 'size': 0.1585}]},
 {'symbol_id': 'LUNO_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T23:55:30.6872278Z',
  'time_coinapi': '2020-07-17T23:55:30.6872278Z',
  'asks': [{'price': 164318.0, 'size': 1e-06}],
  'bids': [{'price': 164046.0, 'size': 0.02003}]},
 {'symbol_id': 'LUNO_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T23:56:52.8443586Z',
  'time_coinapi': '2020-07-17T23:56:52.8443586Z',
  'asks': [{'price': 164318.0, 'size': 1e-06}],
  'bids': [{'price': 164074.0, 'size': 0.02003}]},
 {'symbol_id': 'LUNO_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T23:58:15.1198622Z',
  'time_coinapi': '2020-07-17T23:58:15.1198622Z',
  'asks': [{'price': 164318.0, 'size': 1e-06}],
  'bids': [{'price': 164021.0, 'size': 0.105196}]},
 {'symbol_id': 'LUNO_SPOT_BTC_ZAR',
  'time_exchange': 

In [44]:
historical_data_33d_dct['BINANCE_SPOT_BTC_ZAR'][-5:]


[{'symbol_id': 'BINANCE_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T06:24:45.8710000Z',
  'time_coinapi': '2020-07-17T06:24:45.9499679Z',
  'asks': [{'price': 165021.0, 'size': 0.047583}],
  'bids': [{'price': 164320.0, 'size': 0.0132}]},
 {'symbol_id': 'BINANCE_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T06:25:48.8070000Z',
  'time_coinapi': '2020-07-17T06:25:48.8859379Z',
  'asks': [{'price': 165021.0, 'size': 0.03624}],
  'bids': [{'price': 164320.0, 'size': 0.0132}]},
 {'symbol_id': 'BINANCE_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T06:26:59.4480000Z',
  'time_coinapi': '2020-07-17T06:26:59.5277317Z',
  'asks': [{'price': 165732.0, 'size': 0.088497}],
  'bids': [{'price': 164320.0, 'size': 0.0132}]},
 {'symbol_id': 'BINANCE_SPOT_BTC_ZAR',
  'time_exchange': '2020-07-17T06:28:10.0900000Z',
  'time_coinapi': '2020-07-17T06:28:10.1669694Z',
  'asks': [{'price': 165730.0, 'size': 0.082497}],
  'bids': [{'price': 164321.0, 'size': 0.005588}]},
 {'symbol_id': 'BINANCE_SPOT_BTC_ZA

It looks like Binance was cut due to the limit just on last day morning - so to make it easier to analyse we delete the last day

Now change the format so that it can be easily used with pandas

In [50]:
useful_symbol_id_list = ['LUNO_SPOT_BTC_ZAR', 'BINANCE_SPOT_BTC_ZAR']
formatted_data_to_save = {}

for symbol_id in useful_symbol_id_list:
    formatted_data_to_save[symbol_id] = []
    for e in historical_data_33d_dct[symbol_id]:
        elem_dct = {
            'time': e['time_exchange'],
            'asks_price': e['asks'][0]['price'],
            'asks_size': e['asks'][0]['size'],
            'bids_price': e['bids'][0]['price'],
            'bids_size': e['bids'][0]['size']
        }

        # make sure we have consistent data in terms of time for both Luno and Binance
        # exclude anything on 2020-07-17
        # this is the only day where we did not get all data for Binance
        if '2020-07-17' not in elem_dct['time']:
            formatted_data_to_save[symbol_id].append(elem_dct)

Save the data as json:
{'LUNO_SPOT_BTC_ZAR': \[{}, ..., {}\], 'BINANCE_SPOT_BTC_ZAR': \[{}, ..., {}\]}

In [57]:
with open('luno_binance_spot_btc_zar.json', 'w') as f:
    json.dump(formatted_data_to_save, f)

Now an example of how to use the data with pandas

In [58]:
import pandas as pd

In [60]:
with open('luno_binance_spot_btc_zar.json', 'r') as f:
    luno_binance_data = json.load(f)

luno_btc_zar_data = pd.DataFrame(luno_binance_data['LUNO_SPOT_BTC_ZAR'])
binance_btc_zar_data = pd.DataFrame(luno_binance_data['BINANCE_SPOT_BTC_ZAR'])

In [61]:
luno_btc_zar_data.head()

,time,asks_price,asks_size,bids_price,bids_size
0,2020-06-15T00:02:19.5214332Z,170386.0,0.534161,170385.0,0.076187
1,2020-06-15T00:02:34.7003021Z,170386.0,0.455860,170385.0,2.500000
2,2020-06-15T00:03:45.2280492Z,170386.0,0.534161,170385.0,1.949240
3,2020-06-15T00:04:36.1349999Z,170386.0,0.455860,170385.0,2.500000
4,2020-06-15T00:05:10.9274555Z,170386.0,0.534161,170385.0,2.500000


In [62]:
binance_btc_zar_data.head()

,time,asks_price,asks_size,bids_price,bids_size
0,2020-06-15T00:00:13.4120000Z,171542.0,0.089691,170523.0,0.003790
1,2020-06-15T00:00:21.0070000Z,171542.0,0.089691,170523.0,0.003790
2,2020-06-15T00:00:27.5090000Z,171542.0,0.089691,170523.0,0.003790
3,2020-06-15T00:00:28.5090000Z,171542.0,0.089691,170523.0,0.003790
4,2020-06-15T00:00:29.8080000Z,171542.0,0.089691,170514.0,0.117292
